# **Решение тестового задания KazanExpress**


Поставленная задача:

"На нашем маркетплейсе ежедневно появляются сотни новых товаров. Однако, проверить правильность заполнения информации обо всех товарах сразу невозможно. Неверно определенная категория зачастую приводит к потенциально упущенной прибыли как со стороны продавца, так и со стороны маркетплейса. Мы хотим научиться предсказывать категорию на основе описания, картинки и других параметрах товаров."

# *Import библиотек и загрузка данных из zip архива*

In [ ]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 36.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=c0338656cc970045cc2ee46768cd61212f17f030f395bb54ce05e5862e64e3ca
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
import re
import pymorphy2
import nltk
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import LabelEncoder

In [ ]:
!unzip /content/drive/MyDrive/internship_2023.zip -d my_data

*Загрузка тренировочных данных*

In [ ]:
train_data = pd.read_parquet('/content/my_data/train.parquet').reset_index(drop=True)

In [ ]:
le = LabelEncoder()
train_data['category_id'] = le.fit_transform(train_data.category_id)

In [ ]:
train_data.head()

,product_id,category_id,sale,shop_id,shop_title,rating,text_fields,category_name
0,325286,251,False,9031,Aksik,5.0,"{""title"": ""Зарядный кабель Borofone BX1 Lightn...",Все категории->Электроника->Смартфоны и телефо...
1,888134,748,False,18305,Sela,5.0,"{""title"": ""Трусы Sela"", ""description"": ""Трусы-...",Все категории->Одежда->Женская одежда->Белье и...
2,1267173,560,False,16357,ЮНЛАНДИЯ канцтовары,5.0,"{""title"": ""Гуашь \""ЮНЫЙ ВОЛШЕБНИК\"", 12 цветов...",Все категории->Хобби и творчество->Рисование->...
3,1416943,61,False,34666,вася-nicotine,4.0,"{""title"": ""Колба для кальяна Крафт (разные цве...",Все категории->Хобби и творчество->Товары для ...
4,1058275,413,False,26389,Lim Market,4.6,"{""title"": ""Пижама женская, однотонная с шортам...",Все категории->Одежда->Женская одежда->Домашня...


# *Анализ данных*

In [ ]:
train_data.loc[0], train_data.loc[567]

(product_id                                                  325286
 category_id                                                    251
 sale                                                         False
 shop_id                                                       9031
 shop_title                                                   Aksik
 rating                                                         5.0
 text_fields      {"title": "Зарядный кабель Borofone BX1 Lightn...
 category_name    Все категории->Электроника->Смартфоны и телефо...
 Name: 0, dtype: object,
 product_id                                                 1768203
 category_id                                                    655
 sale                                                         False
 shop_id                                                      12630
 shop_title                                                   Весна
 rating                                                         5.0
 text_fields      {"tit

Видим, что информацию о категории товара можем получить из полей shop_title и text_fields. Информация об id товара, id магазина и скидке не будут значимо влиять на его категорию.

In [ ]:
train_data.text_fields[0], train_data.text_fields[7869]

('{"title": "Зарядный кабель Borofone BX1 Lightning для айфон, 1м", "description": "<p><span style=\\"background-color: transparent; color: rgb(0, 26, 52);\\">Зарядный кабель Borofone BX1 подходит для зарядки всех гаджетов и аксессуаров с разъемом </span>Lightning.</p><p><span style=\\"color: rgb(0, 26, 52); background-color: transparent;\\">Поддерживает быструю зарядку.</span></p><p><span style=\\"color: rgb(0, 26, 52); background-color: transparent;\\">Подходит для передачи данных.</span></p>", "attributes": ["Длина: 1м", "Разъем: Lightning", "Подерживает быструю зарядку", "Максимальный ток: 2.0А", "Для зарядки и синхронизации данных", "Вес: 22 г."], "custom_characteristics": {}, "defined_characteristics": {"Цвет": ["Черный", "Белый"]}, "filters": {"Цвет": ["Белый", "Черный"]}}',
 '{"title": "Магнит \\"Волшебного Нового Года!\\" символ года, подарки", "description": "<p>Магнит \\"Волшебного Нового Года!\\" символ года, подарки</p><p><br></p>", "attributes": ["Индивидуальная упаковка 

Поле text_fields обладает наиболее значимой информацией для определения категории товара. Но видим, что в нем много "шума". Для каждого товара в этом поле присутствуют слова title, description и т.д., есть много специальных символов в виде скобок, двоеточий и запятых. Так же, посмотрев для бОльшего количества товаров это поле, можно сделать предположение, что слова на английском языке не будут нести важной общей информации. Во многих встречается обозначение цвета как rgb(-, -, -), background-color и похожие слова, не имеющие значения для предсказания категории товара по этому полю.

Поэтому можно предположить, что для анализа поля text_field будет достаточно анализировать слова, которые содержат только русские буквы.

Можем удалить из поля text_fields все символы кроме букв.

Посмотрим, можем ли удалить английские буквы. Для этого посмотрим на text_fields у тех товаров, где количество английских букв больше количества русских. Заранее уберем title, description и т.д.

In [ ]:
#a - 97, z - 122, A- 65, Z - 90, а - 1072, я - 1103, А - 1040, Я - 1071
def count_russian_english_letters(s):
  count_ru = 0
  count_en = 0
  for i in s:
    if (1072 <= ord(i) <= 1103) or (1040 <= ord(i) <= 1071):
      count_ru += 1
    if (97 <= ord(i) <= 122) or (65 <= ord(i) <= 90):
      count_en += 1
  return count_ru, count_en
def delete(s):
  s = s.replace('title', '')
  s = s.replace('description', '')
  s = s.replace('attributes', '')
  s = s.replace('custom_characteristics', '')
  s = s.replace('defined_characteristics', '')
  s = s.replace('filters', '')
  s = s.replace('rgb', '')
  return s
flags = 0
strs = train_data.text_fields
count = 10
for i in strs:
  i = delete(i)
  count_ru, count_en = count_russian_english_letters(i)
  if count_en > count_ru:
    count -= 1
    print(i)
    flags += 1
  if count == 0:
    break
flags

{"": "Бюстгальтер", "": "<p><img src=\"https://image-cdn.kazanexpress.ru/bouih64h5i5fc12fkm4g/original.jpg\"><img src=\"https://image-cdn.kazanexpress.ru/bp4i5path9ddr4mbr0d0/original.jpg\"><img src=\"https://image-cdn.kazanexpress.ru/bouih9gsmgfmei8hp8m0/original.jpg\"><a href=\"https://kazanexpress.ru/avasecret\" target=\"_blank\"><img src=\"https://image-cdn.kazanexpress.ru/bp1sqech5i5fc12fn9t0/original.jpg\"><img src=\"https://image-cdn.kazanexpress.ru/bouihl8smgfmei8hp8mg/original.jpg\"></a></p><p><a href=\"https://kazanexpress.ru/avasecret\" target=\"_blank\"><img src=\"https://image-cdn.kazanexpress.ru/bovej62th9d5b5domaeg/original.jpg\"></a><a href=\"https://wa.me/79996720989?text=%D0%97%D0%B4%D1%80%D0%B0%D0%B2%D1%81%D1%82%D0%B2%D1%83%D0%B9%D1%82%D0%B5,%20%D1%83%20%D0%BC%D0%B5%D0%BD%D1%8F%20%D1%82%D0%B0%D0%BA%D0%BE%D0%B9%20%D0%B2%D0%BE%D0%BF%D1%80%D0%BE%D1%81:\" target=\"_blank\"><img src=\"https://image-cdn.kazanexpress.ru/bouihq2th9d5b5dolgqg/original.jpg\"></a></p>", "": [" 

10

Видим, что большее количестсво английских слов - не такая важная информация (например, ссылки и специальная информация для них), как слова на русском. Так же можем еще раз убедиться, что можно оставить только буквы, Так как в text_fields используется много цифр для описания модели товара и его параметров, что не важно для категории.

Удалим все символы не из русского алфивита. Уберем стоп-слова и приведем слова к нормальной форме.

In [ ]:
train_data.shop_title[0], train_data.shop_title[687], train_data.shop_title[2], train_data.shop_title[5687]

('Aksik', 'MIR', 'ЮНЛАНДИЯ канцтовары', 'fceVdom')

Аналогичные преобразования можно сделать с полем shop_title, т.к. там так же могут быть слова на английском, которые не помогут определить категорию (особенно после удаления английских букв из text_fields), но могут быть русские слова, которое имеют значение. 

# *Функции для обработки текстов*

In [ ]:
nltk.download('stopwords')
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token)
    if len(tokens) > 2:
        return tokens
    return None

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def create_train_data(data):
  shop_titles = np.array(data.shop_title)
  all_text_fields = np.array(data.text_fields)
  x_train = shop_titles + " " + all_text_fields
  for i in range(len(x_train)):
    x_train[i] = re.sub('[^а-яА-Я ]', '', x_train[i])
    x_train[i] = re.sub(r'\b\w{1}\b', '', x_train[i])
    x_train[i] = lemmatize(x_train[i])
  return x_train

In [ ]:
def get_preprocessed_data(data):
  x_train = create_train_data(data)
  for i in range(len(x_train)):
    if x_train[i]:
      x_train[i] = ' '.join(x_train[i])
    else:
      x_train[i] = ''
  return x_train

# Берем из таблицы нужные данные для x_train, применяем для них TfidfVectorizer, берем нужные данные для y_train

In [ ]:
x_train_preprocessed = get_preprocessed_data(train_data)

In [ ]:
def write_data_to_file(data, filename_pkl, filename_serialized):
  with open(filename_pkl, 'wb') as file:
      pickle.dump(data, file)
  serialized = pickle.dumps(data)
  filee = open(filename_serialized, 'wb')
  filee.write(serialized)
  filee.close()

In [ ]:
def get_data_from_file(filename_serialized):
  f = open(filename_serialized, 'rb')
  ser = f.read()
  data = pickle.loads(ser)
  return data

In [ ]:
write_data_to_file(x_train_preprocessed, 'x_train_preprocessed2103.pkl', 'serialized_x_train_preprocessed2103')

In [ ]:
tf_idf = TfidfVectorizer(max_df=0.8, min_df=10)
tf_idf.fit(x_train_preprocessed)
x_train_preprocessed = tf_idf.transform(x_train_preprocessed)

In [ ]:
y_train = train_data.category_id

In [ ]:
svm_model = LinearSVC(C=0.5, tol=1e-05, max_iter=100, random_state=42)

# Обучение модели, сохранение, предсказание на тестовых данных, сохранение предсказания

In [ ]:
svm_model.fit(x_train_preprocessed, y_train)

In [ ]:
write_data_to_file(svm_model, 'svm_model2103.sav', 'serialized_svm_model2103')

In [ ]:
test_data = pd.read_parquet('/content/my_data/test.parquet').reset_index(drop=True)

In [ ]:
x_test_preprocessed = get_preprocessed_data(test_data)
x_test_preprocessed = tf_idf.transform(x_test_preprocessed)

In [ ]:
y_pred = svm_model.predict(x_test_preprocessed)

In [ ]:
write_data_to_file(y_pred, 'y_pred2103.pkl', 'serialized_y_pred2103')

Это итоговое предсказание модели.

In [ ]:
product_ids = test_data.product_id
d = {'product_id': np.array(product_ids), 'predicted_category_id': np.array(y_pred)}
result = pd.DataFrame(data=d)
res = result.to_parquet('result.parquet')

# **Описание других исследований**

До обучения с помощью LinearSVC были обучены RandomForestClassifier и KNeighborsClassifier. Качество на них по кросс-валлидации было примерно 0.67, в то время как для LinearSVC оно было около 0.87.

Пример того, как выполнялась кросс-валидация:


In [ ]:
from sklearn.model_selection import cross_val_score
svm_val = LinearSVC(C=0.5, tol=1e-05, max_iter=100, random_state=42)
cross_val_score(svm_val, x_train_preprocessed, y_train, cv=3, n_jobs=4)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


array([0.84361625, 0.84282093, 0.84338063])

Так же был подбор параметров с помощью GridSearchCV.

Были попытки работы с картинками. Попыталась обучить нейросеть tf.keras.applications.ResNet152 на тренировачном датасете, добавив туда слой для классификации на заданные в тренировочном датасете классы. Однако качество модели получалось плохим, около 0.41... Код обучения нейросети представлен ниже.

# Обучение tf.keras.applications.ResNet152

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model

In [ ]:
imgs = []
y_train = []
product_ids = np.array(train_data.product_id)
for i in range(len(train_data.product_id)):
  if (i % 2 == 0):
    img = tf.convert_to_tensor(tf.keras.utils.load_img('/content/my_data/images/train/' + str(product_ids[i])+ '.jpg'))
    img = tf.image.resize(img, (64, 64), method='nearest')
    imgs.append(img)
    y_train.append(train_data.category_id[i])

In [ ]:
imgs_val = []
product_ids = np.array(train_data.product_id)
y_train_val = []
for i in range(346, len(train_data.product_id)):
  if i % 2 != 0 and i % 3 != 0:
    img = tf.convert_to_tensor(tf.keras.utils.load_img('/content/my_data/images/train/' + str(product_ids[i])+ '.jpg'))
    img = tf.image.resize(img, (64, 64), method='nearest')
    imgs_val.append(img)
    y_train_val.append(train_data.category_id[i])

In [ ]:
base_model = tf.keras.applications.ResNet152(weights = 'imagenet', include_top = False, input_shape = (64,64,3))
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(1000, activation='relu')(x)
predictions = layers.Dense(874, activation = 'softmax')(x)

In [ ]:
head_model = Model(inputs = base_model.input, outputs = predictions)
head_model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy']) #Здесь надо было бы f1_score, заметила только после долгого обучения...

In [ ]:
history = head_model.fit(np.array(imgs), np.array(y_train), batch_size=256, epochs=8, validation_data=(np.array(imgs_val), np.array(y_train_val)))

Epoch 1/8
178/178 [==============================] - 3403s 19s/step - loss: 4.8554 - accuracy: 0.1964 - val_loss: 4.0970 - val_accuracy: 0.2483
Epoch 2/8
178/178 [==============================] - 3348s 19s/step - loss: 3.4961 - accuracy: 0.3027 - val_loss: 3.8566 - val_accuracy: 0.2779
Epoch 3/8
178/178 [==============================] - 3329s 19s/step - loss: 2.8482 - accuracy: 0.3833 - val_loss: 3.9182 - val_accuracy: 0.2858
Epoch 4/8
178/178 [==============================] - ETA: 0s - loss: 2.2952 - accuracy: 0.4664 

# Что можно сделать дальше

Если бы качество tf.keras.applications.ResNet152 получилось бы лучше, то была бы обучена новая модель, сверху LinearSVC, обученной выше на текстах, и tf.keras.applications.ResNet152.

Поэтому можно улучшить качество нейросети, возможно, выбрать другую архитектуру. Затем взять предсказания LinearSVC и tf.keras.applications.ResNet152, конкатерировать их и взять их в качестве x_train, а y_train оставить то же и обучить, например, RandomForest.

Так же для улучшения ккачества можно попробовать более внимательно анализировать данные. Например, обратить внимание, что в text_fields в title хранится более важная и компактная информация, чем в description.

# *Литература*

https://python-school.ru/blog/russian-text-preprocessing/

https://medium.com/swlh/resnet-with-tensorflow-transfer-learning-13ff0773cf0c